### TO DO
- Add images for each card - https://pkmncards.com/card/alakazam-legendary-collection-lc-1/
- Card condition handling (shadowless, gym leaders, apostrophes/periods in name, ...)
- integrate [pokellector](https://www.pokellector.com/card/Pikachu-VMAX-Vivid-Voltage-SWSH4-188) for "detailed search"

### Card condition handling
There are many different "types" of cards, including but not limited to: first edition, shadowless, mega, ex, gym leaders (with apostophes/periods in the name). All these will have to be cased for in order for us to hit the right endpoint with our function. Below is a list of all card "types" -> and how the url "appears" on price charting

##### Typical
- Card #_ -> /card-_
- Card [1st Edition] #_ -> /card-1st-edition-_
- Card [Shadowless] #_ -> /card-shadowless-_
- Car'd #_ -> /car%27d-#
- Ca.rd #_ -> /ca-rd-#

##### Atypical
- Card [Trainer Deck A/B] #_ (base set)
- Card [E3 Red Cheeks] #_ (base set pikachu)
- Card [1st Edition Red Cheeks] #_ (base set pikachu)
- Card [Shadowless Red Cheeks] #_ (base set pikachu)
- Card [Black Flame] #_ (base set ninetales)
- Card [Gold Border] #_ (jungle meowth)
- Card [W Stamp] #_ (jungle pikachu, base set 2 wartortle, team rocket dark charmeleon, ...)
- Card [Prerelease] #_ (fossil aerodactyl)
- Card [Prerelease Holo] #_ (team rocket dark gyarados)

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')

Import set and card count excel

In [6]:
df = pd.read_excel('Set_and_Card_Ct.xlsx', index_col=0)
df.head()

,Set Name,Card Count
0,Base Set,102
1,Jungle,64
2,Fossil,62
3,Base Set 2,130
4,Team Rocket,82


### Define the card input function - quick mode!

In [7]:
card_db = pd.DataFrame()

def card_value():
    tmp_row = pd.DataFrame()  # to be appended to card_db
    
    card_name_full = input("Enter the Pokemon name EXACTLY as shown (ex. Charizard ex): ")
    card_and_set_num = input("Enter the card number and set number found at the bottom of the card \
                        Exactly as shown (ex. 4/130): ")
#     card_feats = input("Does the card have any special features - EX, G LV X, First Edition, Shadowless? ")
#     card_mega = input("Is the card a Mega Evolution? ")
#     return card_name_full, card_and_set_num #, card_feats, card_mega
    card_num = card_and_set_num.split('/')[0]
    set_num = int(card_and_set_num.split('/')[1])
    
    # determine which set(s) correspond to the the card count
    potential_sets = [el.replace(' ','-') for el in list(df[df['Card Count'] == set_num]['Set Name'])]

    url_list = [f"https://www.pricecharting.com/game/pokemon-{set_}/{card_name_full}-{card_num}"
                for set_ in potential_sets]
    for url in url_list:
        r = requests.get(url, verify=False)
        soup = BeautifulSoup(r.text, "html.parser")
        try:
            price = [el.text.strip() for el in soup.select('#used_price > .js-price')][0]
        except: pass
    print(f'Your {card_name_full.title()}, ungraded, is worth up to {price}!')
    
#     # append to card db
#     tmp_row['Card Name'] = card_name_full.title()
#     tmp_row['Card and Set Number'] = card_and_set_num
#     tmp_row['Est. Ungraded Price'] = f'${price}'
#     card_db.append(tmp_row)

In [8]:
card_value()

Enter the Pokemon name EXACTLY as shown (ex. Charizard ex): lugia
Enter the card number and set number found at the bottom of the card                         Exactly as shown (ex. 4/130): 9/111
Your Lugia, ungraded, is worth up to $131.10!


In [ ]:
'''
entry: charizard 4/102 first edition ...
check: 102 could be base set, ...
    - is charizard 4 in base set, other set, ...?
    - YES it's in base set, use base set url to find price on pricecharting
    - NO, invalid entry
    
    - selenium to go to pricecharting/<set name> and pull in each card for a set. 
    "Charizard #4 shadowless", "Mewtwo #10", ... and do a left join on the set name
'''